In [1]:
import tensorflow as tf
from tensorflow.python.ops.rnn import _transpose_batch_time

from datetime import datetime
import numpy as np
import pandas as pd
import glob, os, sys, math, warnings, copy, time
import matplotlib.pyplot as  plt
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

# customized ftns 
from preprocessing import *
from utilities import *
from model import *
from train import train_all_single_policies
# ---------------------------------------------------------
%matplotlib inline
%load_ext autoreload
%autoreload 2
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')
# ---------------------------------------------------------
# directories
main_dir = '../'
game_dir = main_dir+'data/'
Data = LoadData(main_dir, game_dir)
models_path = './models/'

C:\Users\sshi\AppData\Local\Continuum\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#### Load raw data

In [2]:
# %%time
game_id = '0021500196'
game_data = Data.load_game(game_id)
events_df = pd.DataFrame(game_data['events'])
print('raw events shape:', events_df.shape)
home_id = events_df.loc[0].home['teamid']
events_df.head(3)

raw events shape: (234, 8)


,end_time_left,home,moments,orig_events,playbyplay,quarter,start_time_left,visitor
0,716.37,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224810037, 720.0, 24.0, None, [[-1, -...",[0],"Empty DataFrame Columns: [GAME_ID, EVENTNUM, E...",1,720.00,"{'abbreviation': 'TOR', 'players': [{'playerid..."
1,701.37,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224810037, 720.0, 24.0, None, [[-1, -...","[1, 2]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,720.00,"{'abbreviation': 'TOR', 'players': [{'playerid..."
2,682.36,"{'abbreviation': 'LAC', 'players': [{'playerid...","[[1, 1448224825036, 707.33, 11.37, None, [[-1,...","[3, 4]",GAME_ID EVENTNUM EVENTMSGTYPE EVENTMS...,1,707.33,"{'abbreviation': 'TOR', 'players': [{'playerid..."


#### Get some suplementary data

In [3]:
# # play id to play roles/positions
# id_role = id_position(events_df)
# check_game_roles_duplicates(id_role)

# # its possible that F has similar role as G-f or F-G, we create empty slots to ensure meta order
# # ddentify defending and offending runs (this is included in process_moments)
# court_index = Data.load_csv('./meta_data/court_index.csv')
# court_index = dict(zip(court_index.game_id, court_index.court_position))

# # home and visitor ids
# homeid = events_df.loc[0].home['teamid']
# awayid = events_df.loc[0].visitor['teamid']

### FILTER EVENTS

In [4]:
# events_df.loc[3].playbyplay.to_dict('list')

In [5]:
# events_df.moments[3]

In [6]:
# n_event = 233
# P = PlotGame('0021500196', main_dir, game_dir)
# for i in range(len(events_df.moments[n_event])):
#     P.load_moment2img(game_data, n_event, i)

### as we saw that the playbyplay description of events is not accurate, so for now at least we will not try to filter by events

In [7]:
def remove_non_eleven(events_df, event_length_th=25, verbose=False):
    df = events_df.copy()
    home_id = df.loc[0]['home']['teamid']
    away_id = df.loc[0]['visitor']['teamid']
    def remove_non_eleven_(moments, event_length_th=25, verbose=False):
        ''' Go through each moment, when encounters balls not present on court,
            or less than 10 players, discard these moments and then chunk the following moments 
            to as another event.

            Motivations: balls out of bound or throwing the ball at side line will
                probably create a lot noise for the defend trajectory learning model.
                We could add the case where players are less than 10 (it could happen),
                but this is not allowed in the model and it requres certain input dimension.

            moments: A list of moments
            event_length_th: The minimum length of an event

            segments: A list of events (or, list of moments) e.g. [ms1, ms2] where msi = [m1, m2]
        '''

        segments = []
        segment = []
        # looping through each moment
        for i in range(len(moments)):
            # get moment dimension
            moment_dim = len(moments[i][5])
            # 1 bball + 10 players
            if moment_dim == 11:
                segment.append(moments[i])
            # less than ten players or basketball is not on the court
            else:
    #             print('less than 11')
                # only grab these satisfy the length threshold
                if len(segment) >= event_length_th:
                    segments.append(segment)
                # reset the segment to empty list
                segment = []
        # grab the last one
        if len(segment) >= event_length_th:
            segments.append(segment)
        if len(segments) == 0:
            if verbose: print('Warning: Zero length event returned')
        return segments
    # process for each event (row)
    df['chunked_moments'] = df.moments.apply(lambda m: remove_non_eleven_(m, event_length_th, verbose))
    # in case there's zero length event
    df = df[df['chunked_moments'].apply(lambda e: len(e)) != 0]
    df['chunked_moments'] = df['chunked_moments'].apply(lambda e: e[0])
    return df['chunked_moments'].values, {'home_id': home_id, 'away_id': away_id}

In [8]:
r, team_ids = remove_non_eleven(events_df)

In [9]:
type(r[0]) == list

True

In [10]:
events_df1 = pd.DataFrame({'moments': r})

In [11]:
def chunk_shotclock(events_df, event_length_th=25, verbose=False):
    df = events_df.copy()
    def chunk_shotclock_(moments, event_length_th, verbose):
        ''' When encounters ~24secs or game stops, chunk the moment to another event.
            shot clock test:
            1) c = [20.1, 20, 19, None,18, 12, 9, 7, 23.59, 23.59, 24, 12, 10, None, None, 10]
              result = [[20.1, 20, 19], [18, 12, 9, 7], [23.59], [23.59], [24, 12, 10]]
            2) c = [20.1, 20, 19, None, None,18, 12, 9, 7, 7, 7, 23.59, 23.59, 24, 12, 10, None, None, 10]
              result = [[20.1, 20, 19], [18, 12, 9, 7], [7], [7], [23.59], [23.59], [24, 12, 10]]

            Motivations: game flow would make sharp change when there's 24s or 
            something happened on the court s.t. the shot clock is stopped, thus discard
            these special moments and remake the following valid moments to be next event.

            moments: A list of moments
            event_length_th: The minimum length of an event
            verbose: print out exceptions or not

            segments: A list of events (or, list of moments) e.g. [ms1, ms2] where msi = [m1, m2] 
        '''

        segments = []
        segment = []
        # naturally we won't get the last moment, but it should be okay
        for i in range(len(moments)-1):
            current_shot_clock_i = moments[i][3]
            next_shot_clock_i = moments[i+1][3]
            # sometimes the shot clock value is None, thus cannot compare
            try:
                # if the game is still going i.e. sc is decreasing
                if next_shot_clock_i < current_shot_clock_i:
                    segment.append(moments[i])
                # for any reason the game is sstopped or reset
                else:
                    # not forget the last moment before game reset or stopped
                    if current_shot_clock_i < 24.:
                        segment.append(moments[i])
                    # add length condition
                    if len(segment) >= event_length_th:
                        segments.append(segment)
                    # reset the segment to empty list
                    segment = []
            # None value
            except Exception as e:
                if verbose: print(e)
                # not forget the last valid moment before None value
                if current_shot_clock_i != None:
                    segment.append(moments[i])    
                if len(segment) >= event_length_th:
                    segments.append(segment)
                # reset the segment to empty list
                segment = []

        # grab the last one
        if len(segment) >= event_length_th:
            segments.append(segment)            
        if len(segments) == 0:
            if verbose: print('Warning: Zero length event returned')
        return segments
    
    # process for each event (row)
    df['chunked_moments'] = df.moments.apply(lambda m: chunk_shotclock_(m, event_length_th, verbose))
    # in case there's zero length event
    df = df[df['chunked_moments'].apply(lambda e: len(e)) != 0]
    df['chunked_moments'] = df['chunked_moments'].apply(lambda e: e[0])
    return df['chunked_moments'].values

In [12]:
r1 = chunk_shotclock(events_df1)
events_df2 = pd.DataFrame({'moments': r1})

In [13]:
def chunk_halfcourt(events_df, event_length_th=25, verbose=False):
    df = events_df.copy()
    def chunk_halfcourt_(moments, event_length_th, verbose):
        ''' Discard any plays that are not single sided. When the play switches 
            court withhin one event, we chunk it to be as another event
        '''

        # NBA court size 94 by 50 feet
        half_court = 94/2. # feet
        cleaned = []

        # remove any moments where two teams are not playing at either side of the court
        for i in moments:
            # the x coordinates is on the 3rd or 2 ind of the matrix,
            # the first and second is team_id and player_id
            team1x = np.array(i[5])[1:6, :][:, 2]    # player data starts from 1, 0 ind is bball
            team2x = np.array(i[5])[6:11, :][:, 2]
            # if both team are on the left court:
            if sum(team1x <= half_court)==5 and sum(team2x <= half_court)==5:
                cleaned.append(i)
            elif sum(team1x >= half_court)==5 and sum(team2x >= half_court)==5:
                cleaned.append(i)

        # if teamns playing court changed during same list of moments,
        # chunk it to another event
        segments = []
        segment = []
        for i in range(len(cleaned)-1):
            current_mean = np.mean(np.array(cleaned[i][5])[:, 2], axis=0)
            current_pos = 'R' if current_mean >= half_court else 'L'
            next_mean = np.mean(np.array(cleaned[i+1][5])[:, 2], axis=0)
            next_pos = 'R' if next_mean >= half_court else 'L'

            # the next moment both team are still on same side as current
            if next_pos == current_pos:
                segment.append(cleaned[i])
            else:
                if len(segment) >= event_length_th:
                    segments.append(segment)
                segment = []
        # grab the last one
        if len(segment) >= event_length_th:
            segments.append(segment)            
        if len(segments) == 0:
            if verbose: print('Warning: Zero length event returned')
        return segments
    
    # process for each event (row)
    df['chunked_moments'] = df.moments.apply(lambda m: chunk_halfcourt_(m, event_length_th, verbose))
    # in case there's zero length event
    df = df[df['chunked_moments'].apply(lambda e: len(e)) != 0]
    df['chunked_moments'] = df['chunked_moments'].apply(lambda e: e[0])
    return df['chunked_moments'].values

In [14]:
r2 = chunk_halfcourt(events_df2)
events_df3 = pd.DataFrame({'moments': r2})

In [15]:
# court_index

In [16]:
court_index = pd.read_csv('./meta_data/court_index.csv')
court_index = dict(zip(court_index.game_id, court_index.court_position))
court_index[int('0021500196')]

1

In [17]:
def reorder_teams(events_df, game_id):
    df = events_df.copy()
    def reorder_teams_(input_moments, game_id):
        ''' 1) the matrix always lays as home top and away bot VERIFIED
            2) the court index indicate which side the top team (home team) defends VERIFIED

            Reorder the team position s.t. the defending team is always the first 

            input_moments: A list moments
            game_id: str of the game id
        '''
        # now we want to reorder the team position based on meta data
        court_index = pd.read_csv('./meta_data/court_index.csv')
        court_index = dict(zip(court_index.game_id, court_index.court_position))

        full_court = 94.
        half_court = full_court/2. # feet
        home_defense = court_index[int(game_id)]
        moments = copy.deepcopy(input_moments)
        for i in range(len(moments)):
            home_moment_x = np.array(moments[i][5])[1:6,2]
            away_moment_x = np.array(moments[i][5])[6:11,2]
            quarter = moments[i][0]
            # if the home team's basket is on the left
            if home_defense == 0:
                # first half game
                if quarter <= 2:
                    # if the home team is over half court, this means they are doing offense
                    # and the away team is defending, so switch the away team to top
                    if sum(home_moment_x>=half_court)==5 and sum(away_moment_x>=half_court)==5:
                        moments[i][5][1:6], moments[i][5][6:11] = moments[i][5][6:11], moments[i][5][1:6]
                        for l in moments[i][5][1:6]:
                            l[2] = full_court - l[2]
                        for l in moments[i][5][6:11]:
                            l[2] = full_court - l[2]
                # second half game      
                elif quarter > 2: # second half game, 3,4 quarter
                    # now the home actually gets switch to the other court
                    if sum(home_moment_x<=half_court)==5 and sum(away_moment_x<=half_court)==5:
                        moments[i][5][1:6], moments[i][5][6:11] = moments[i][5][6:11], moments[i][5][1:6]
                    elif sum(home_moment_x>=half_court)==5 and sum(away_moment_x>=half_court)==5:
                        for l in moments[i][5][1:6]:
                            l[2] = full_court - l[2]
                        for l in moments[i][5][6:11]:
                            l[2] = full_court - l[2]
                else:
                    print('Should not be here, check quarter value')
            # if the home team's basket is on the right
            elif home_defense == 1:
                # first half game
                if quarter <= 2:
                    # if the home team is over half court, this means they are doing offense
                    # and the away team is defending, so switch the away team to top
                    if sum(home_moment_x<=half_court)==5 and sum(away_moment_x<=half_court)==5:
                        moments[i][5][1:6], moments[i][5][6:11] = moments[i][5][6:11], moments[i][5][1:6]
                    elif sum(home_moment_x>=half_court)==5 and sum(away_moment_x>=half_court)==5:
                        for l in moments[i][5][1:6]:
                            l[2] = full_court - l[2]
                        for l in moments[i][5][6:11]:
                            l[2] = full_court - l[2]
                # second half game      
                elif quarter > 2: # second half game, 3,4 quarter
                    # now the home actually gets switch to the other court
                    if sum(home_moment_x>=half_court)==5 and sum(away_moment_x>=half_court)==5:
                        moments[i][5][1:6], moments[i][5][6:11] = moments[i][5][6:11], moments[i][5][1:6]
                        for l in moments[i][5][1:6]:
                            l[2] = full_court - l[2]
                        for l in moments[i][5][6:11]:
                            l[2] = full_court - l[2]
                else:
                    print('Should not be here, check quarter value')
        return moments
    return [reorder_teams_(m, game_id) for m in df.moments.values]

In [18]:
home_id

1610612746

In [19]:
court_index[int(game_id)]

1

In [40]:
r3 = reorder_teams(events_df3, game_id)
events_df4 = pd.DataFrame({'moments': r3})

In [21]:
r3[0][0]

[1,
 1448224825557,
 706.81,
 23.99,
 None,
 [[-1, -1, 5.18438, 24.50115, 11.62111],
  [1610612761, 2449, 9.8067, 28.92548, 0.0],
  [1610612761, 201960, 24.3734, 21.43063, 0.0],
  [1610612761, 200768, 6.86594, 20.4174, 0.0],
  [1610612761, 201942, 17.89619, 25.21197, 0.0],
  [1610612761, 202687, 10.1885, 24.11858, 0.0],
  [1610612746, 1718, 34.33032, 10.36747, 0.0],
  [1610612746, 200755, 32.26942, 26.02228, 0.0],
  [1610612746, 101108, 18.86048, 7.87816, 0.0],
  [1610612746, 201599, 35.95785, 20.65878, 0.0],
  [1610612746, 201933, 25.66964, 40.3966, 0.0]]]

In [22]:
m = np.array(r3[0][0][5])

In [23]:
m[0]

array([-1.     , -1.     ,  5.18438, 24.50115, 11.62111])

In [24]:
m[1]

array([1.61061276e+09, 2.44900000e+03, 9.80670000e+00, 2.89254800e+01,
       0.00000000e+00])

In [30]:
# features = np.concatenate((m[1:11, 2:4].reshape(-1), m[0][2:5], np.array([r3[0][0][0]]), np.array([r3[0][0][2]])))#,2]]))

In [31]:
# features

In [32]:
# len(features)

In [37]:
# events_df4.moments.loc[0]

In [92]:
def flatten_moments(events_df):
    df = events_df.copy()
    def flatten_moment(moment):
        m = np.array(moment[5])
        features = np.concatenate((m[1:11, 2:4].reshape(-1),    # x,y of all 10 players 
                                   m[0][2:5],                   # basketball x,y,z 
                                   np.array([moment[0]]),       # quarter number 
                                   np.array([moment[2]]),       # time in seconds left to the end of the period
                                   np.array([moment[3]])))      # shot clock 
        return features
    df['flattened'] = df.moments.apply(lambda ms: [flatten_moment(m) for m in ms])
    return df['flattened'].values

r4 = flatten_moments(events_df4)
events_df5 = pd.DataFrame({'moments': r4})   


def create_static_features(events_df):
    df = events_df.copy()
    def create_static_features_(moment):
        ''' moment: flatten moment i.e. (25=10*2+3+2,)'''
        # distance of each players to the ball
        player_xy = moment[:10*2]
        b_xy = moment[10*2:10*2+2]
        hoop_xy = np.array([3.917, 25])

        def disp_(pxy, target):
            # dispacement to bball
            disp = pxy.reshape(-1, 2) - np.tile(target, (10, 1))
            r = np.sqrt(disp[:,0]**2 + disp[:, 1]**2)               # r 
            cos_theta = disp[:, 0]/r                                # costheta
            sin_theta = disp[:, 1]/r                                # sintheta
            theta = np.arccos(cos_theta)                            # theta
            return np.concatenate((r, cos_theta, sin_theta, theta))
        return np.concatenate((moment, disp_(player_xy, b_xy), disp_(player_xy, hoop_xy)))
    df['enriched'] = df.moments.apply(lambda ms: np.vstack([create_static_features_(m) for m in ms]))
    return df['enriched'].values
    
r5 = create_static_features(events_df5)
events_df6 = pd.DataFrame({'moments': r5})

In [101]:
def create_dynamic_features(events_df, fs):
    df = events_df.copy()
    def create_dynamic_features_(moments, fs):
        ''' moments: (moments length, n existing features)'''
        pxy = moments[:, :23] # get the players x,y and basketball x,y,z coordinates
        next_pxy = np.roll(pxy, -1, axis=0) # get next frame value
        vel = ((next_pxy - pxy)/fs)[:-1, :] # the last velocity is not meaningful
        # when we combine this back to the original features, we shift one done,
        # i.e. [p1, p2, ..., pT] combine [_, p2-p1, ...., pT-pT_1]
        # the reason why we shift is that we don't want to leak next position info
        return np.column_stack([moments[1:, :], vel])
    df['enriched'] = df.moments.apply(lambda ms: create_dynamic_features_(ms, fs))
    return df['enriched'].values


In [104]:
r6 = create_dynamic_features(events_df6, 1/25.)
events_df7 = pd.DataFrame({'moments': r6})

In [108]:
r6[0][0]

array([ 9.64471000e+00,  2.88124100e+01,  2.44642000e+01,  2.11397500e+01,
        6.76971000e+00,  2.05938200e+01,  1.78385400e+01,  2.53595700e+01,
        9.95214000e+00,  2.40786000e+01,  3.46125600e+01,  1.04264400e+01,
        3.25844400e+01,  2.58727500e+01,  1.90134700e+01,  8.11988000e+00,
        3.63512500e+01,  2.07010900e+01,  2.60670000e+01,  4.02710300e+01,
        5.26483000e+00,  2.44984300e+01,  1.15501200e+01,  1.00000000e+00,
        7.06770000e+02,  2.39500000e+01,  6.14766397e+00,  1.94909348e+01,
        4.18457203e+00,  1.26031641e+01,  4.70607398e+00,  3.25470453e+01,
        2.73541559e+01,  2.13841531e+01,  3.13174919e+01,  2.61056543e+01,
        7.12446227e-01,  9.85041004e-01,  3.59625785e-01,  9.97662960e-01,
        9.96012818e-01,  9.01701820e-01,  9.98737087e-01,  6.42935914e-01,
        9.92621634e-01,  7.96845379e-01,  7.01726708e-01, -1.72320108e-01,
       -9.33096616e-01,  6.83272862e-02, -8.92102424e-02, -4.32358448e-01,
        5.02417258e-02, -

##### role alignment

In [110]:
# 10*2 (10 players with x,y) + 3(bball x,y,z) + 1(qtr number) + 1(time left in qtr) + 1(sc) + 
# 10*(4(r,cos,sin,theta)_bball + 4(r, cos, sin, theta)_hoop) + 10*2 (10 players vx, vy) + 3(bball vx,vy,vz)
n_fts = 10*2 + 3 + 1 + 1 + 1 + 10*(4+4) + 10*2 + 3
n_fts

129

In [262]:
def hidden_structure_learning(events_df):
    df = events_df.copy()
    def find_player_ind_(player):
    #     player = 0
        assert player < 10
        pxy_ind = [player*2, player*2+1]
        bball_xy_ind = [2*10, 2*10+1, 2*10+2]
        qtr_ind = [23]
        time_left_ind = [24]
        sc_ind = [25]
        polar_bball_ind = [26+player*4, 26+player*4+1, 26+player*4+2, 26+player*4+3]
        polar_hoop_ind = [66+player*4, 66+player*4+1, 66+player*4+2, 66+player*4+3]
        pvxy_ind = [106+player*2, 106+player*2+1]
        bball_vxy_ind = [126, 127, 128]
        player_ind = np.array(pxy_ind + bball_xy_ind + qtr_ind + time_left_ind + sc_ind + polar_bball_ind \
                     + polar_hoop_ind + pvxy_ind + bball_vxy_ind)
        return player_ind

    def create_hmm_input_(events_df, players):
        event = events_df.moments.values
        X = np.concatenate([np.concatenate([ms[:, find_player_ind(player)] for ms in event], axis=0) for player in players], 
                           axis=0)
        lengths = np.concatenate([[len(ms) for ms in event] for _ in range(len(players))],
                                 axis=0)
        assert len(event[0]) == lengths[0]
        assert len(event[-1]) == lengths[-1]
        return X, lengths
    
    def train_hmm_(players_inds, events_df, verbose=True):
        from hmmlearn import hmm
        assert len(players_inds) == 5 # defend and offend players each are five
        X, lengths = create_hmm_input_(events_df, players=players_inds)
        model = hmm.GaussianHMM(n_components=5, covariance_type='diag', n_iter=50, verbose=verbose)
        model.fit(X, lengths)
        state_sequence = model.predict(X, lengths)
        state_sequence_prob = model.predict_proba(X, lengths) # (n_samples, n_components)
        n_samples, _ = state_sequence_prob.shape
        cmeans = model.means_
        return {'state_sequence': state_sequence.reshape(5, -1), 
                'state_sequence_prob': [state_sequence_prob[i:i+n_samples//5] for i in range(0, n_samples, n_samples//5)], 
                'cmeans': cmeans}
    
    defend_players = list(range(5))
    offend_players = list(range(5, 10))
    moments = events_df.moments.values
    return train_hmm_(defend_players, df, False), [[ms[:, find_player_ind(player)] for ms in moments] for player in defend_players]

    

In [263]:
result, moments = hidden_structure_learning(events_df7)

In [264]:
len(moments)

5

In [258]:
len(moments[4])

164

In [265]:
len(result['state_sequence_prob'])

5

In [266]:
result['state_sequence_prob'][0].shape

(20571, 5)

In [247]:
result['state_sequence_prob'].shape

(102855, 5)

In [259]:
np.sum(result['state_sequence_prob'], axis=1)

array([1., 1., 1., ..., 1., 1., 1.])

In [240]:
result['state_sequence']

array([[4, 4, 4, ..., 0, 0, 0],
       [4, 4, 4, ..., 0, 0, 0],
       [4, 4, 4, ..., 0, 0, 0],
       [2, 2, 2, ..., 3, 3, 3],
       [2, 2, 2, ..., 3, 3, 3]])

In [ ]:
from scipy.optimize import linear_sum_assignment


In [251]:
a = result['state_sequence_prob'].reshape(5, 5, -1)

In [248]:
# 1) minimize the wrong posterior i.e. 1 - posterior
# 2) euclidean distance to the means
# 3) adjusted consine similarity to the means

In [253]:
a.shape

(5, 5, 20571)

In [254]:
for i in range(a.shape[-1]):
    print(sum(a[:, :, i]))

[6.79992684e-14 3.00000000e+00 1.64859565e-15 2.00000000e+00
 3.00000000e+00]
[1.47457866e-09 2.00000000e+00 2.00000000e+00 3.12334860e-14
 2.63572905e-13]
[2.00000000e+00 5.35532650e-71 1.86792053e-17 3.00000000e+00
 2.60987739e-10]
[0.00000000e+00 7.70442873e-12 3.00000000e+00 1.72353994e-14
 1.00000001e+00]
[3.00000000e+00 4.22242906e-14 9.15230887e-21 2.30109832e-16
 9.99999995e-01]
[1.34293899e-16 3.00000000e+00 9.48013296e-17 2.00000000e+00
 3.00000000e+00]
[1.28304531e-12 2.00000000e+00 2.00000000e+00 7.29953723e-14
 2.20564409e-13]
[2.00000000e+00 1.08777031e-65 3.33595778e-19 3.00000000e+00
 2.94208361e-10]
[0.00000000e+00 7.52997616e-12 3.00000000e+00 2.29247225e-14
 1.00000006e+00]
[3.00000000e+00 5.03582187e-14 2.76751382e-20 2.47091729e-16
 9.99999936e-01]
[1.38022382e-16 3.00000000e+00 2.59334863e-16 2.00000000e+00
 3.00000000e+00]
[1.32916962e-12 2.00000000e+00 2.00000000e+00 9.95377045e-14
 2.34762817e-13]
[2.00000000e+00 1.66629983e-60 1.39422328e-18 3.00000000e+00
 7.

[9.99999994e-01 6.53573742e-18 4.91560578e-12 2.99999998e+00
 2.00000000e+00]
[1.00000009e+00 3.00000000e+00 2.99999109e+00 2.00000000e+00
 1.38787208e-58]
[1.99999987e+000 3.06540883e-011 2.00000000e+000 9.94637143e-233
 3.00000000e+000]
[7.27798155e-011 2.91157015e-013 1.96615724e-104 2.27336092e-008
 7.19402850e-013]
[1.00000000e+00 2.00000000e+00 8.91034455e-06 9.04661737e-14
 7.69628890e-10]
[9.99999997e-01 4.79089491e-15 5.02796670e-12 2.99999998e+00
 2.00000000e+00]
[1.00000013e+00 2.99776192e-07 2.99999066e+00 2.00000000e+00
 1.65103628e-56]
[1.99999985e+000 2.99999969e+000 2.00000000e+000 1.06810625e-230
 3.00000000e+000]
[7.72330313e-011 1.00000003e+000 2.54611335e-104 2.36927327e-008
 1.33075889e-012]
[1.00000000e+00 9.99999973e-01 9.34404145e-06 6.06434677e-14
 8.36862792e-10]
[9.99999997e-01 1.47199888e-08 5.69401759e-12 2.99999998e+00
 2.00000000e+00]
[1.00000015e+00 1.63196037e-09 2.99999040e+00 2.00000000e+00
 1.38805031e-53]
[1.99999984e+000 3.00000000e+000 2.00000000e

[4.38828771e-010 1.37073000e-012 2.84581842e-277 5.17014152e-010
 1.61334692e-016]
[3.90321039e-11 2.00000000e+00 9.71360903e-08 1.25630348e-12
 8.16017867e-10]
[2.00000000e+00 6.07886628e-13 4.94514880e-18 2.99999999e+00
 2.00000000e+00]
[9.99999994e-001 2.00000001e+000 2.99999993e+000 1.86521020e+000
 1.40622569e-298]
[2.         0.99999999 2.         0.1347898  3.        ]
[4.38071371e-010 8.40893277e-013 6.62359813e-311 1.14370417e-008
 1.07773170e-016]
[3.72149200e-11 2.00000000e+00 6.76993137e-08 1.19132503e-12
 7.44510865e-10]
[2.00000000e+00 4.51613221e-13 1.04794280e-17 2.99999999e+00
 2.00000000e+00]
[9.99999995e-001 2.00000001e+000 2.99999990e+000 1.94363885e+000
 2.48459234e-310]
[2.         0.99999999 2.         0.05636115 3.        ]
[2.97406909e-010 8.23274892e-013 6.69493634e-301 1.12610953e-008
 7.14705975e-017]
[2.53721559e-11 2.00000000e+00 9.92483479e-08 1.16564760e-12
 5.40924577e-10]
[2.00000000e+00 5.25269667e-13 1.56130710e-17 2.99999999e+00
 2.00000000e+00]
[9.

 8.22646463e-12]
[2.00000000e+00 9.99999999e-01 1.21367027e-10 3.00000000e+00
 8.92901182e-10]
[0.00000000e+00 4.43693499e-13 2.99999995e+00 2.43119282e-17
 1.00000000e+00]
[3.00000000e+00 7.29994824e-09 4.99290243e-08 3.58636981e-16
 1.00000000e+00]
[9.91865007e-17 2.99999999e+00 3.05993162e-12 2.00000000e+00
 3.00000000e+00]
[3.31680767e-09 1.00000000e+00 2.00000000e+00 1.89875242e-18
 7.80410829e-12]
[2.00000000e+00 9.99999999e-01 1.66301171e-10 3.00000000e+00
 8.98553887e-10]
[0.00000000e+00 4.43716960e-13 2.99999994e+00 2.93612645e-17
 1.00000000e+00]
[3.00000000e+00 7.21697010e-09 6.33763400e-08 3.26978708e-16
 1.00000000e+00]
[7.91667238e-17 2.99999999e+00 3.43644112e-12 2.00000000e+00
 3.00000000e+00]
[2.24562232e-09 1.00000000e+00 2.00000000e+00 2.84537471e-18
 7.34458110e-12]
[2.00000000e+00 9.99999999e-01 1.98105464e-10 3.00000000e+00
 9.09926999e-10]
[0.00000000e+00 4.51539127e-13 2.99999994e+00 3.91564968e-17
 1.00000000e+00]
[3.00000000e+00 4.10694629e-09 5.88842427e-08 3

[3.00000000e+00 3.00000000e+00 2.75550657e-16 9.99999999e-01
 3.00000000e+00]
[1.09334067e-14 1.99909558e+00 2.00000000e+00 2.82447019e-10
 6.42536833e-14]
[1.36626203e-13 9.04416962e-04 6.08385180e-26 1.62065174e-11
 2.50070674e-09]
[2.00000000e+00 2.31238492e-13 3.00000000e+00 3.00000000e+00
 2.00000000e+00]
[6.59312409e-012 2.58594947e-012 8.30217612e-023 1.00000000e+000
 2.89179591e-115]
[3.00000000e+00 3.00000000e+00 2.74466013e-16 9.99999999e-01
 3.00000000e+00]
[1.82785811e-14 2.00000000e+00 2.00000000e+00 2.77801346e-10
 4.33081053e-14]
[2.10127716e-13 1.18746907e-09 6.20754862e-26 1.29628706e-11
 2.25158108e-09]
[2.00000000e+00 2.48540849e-13 3.00000000e+00 3.00000000e+00
 2.00000000e+00]
[1.08634606e-011 2.79904446e-012 8.69608587e-023 1.00000000e+000
 5.51889633e-116]
[3.00000000e+00 3.00000000e+00 2.76489180e-16 9.99999999e-01
 3.00000000e+00]
[1.61151384e-14 2.00000000e+00 2.00000000e+00 2.83648686e-10
 2.55146004e-14]
[1.86269744e-13 2.84072883e-10 6.41055204e-26 7.767555

...

[2.00000000e+000 1.42808716e-105 2.99999999e+000 2.73625350e-013
 9.99999261e-001]
[4.06720069e-47 2.99999958e+00 3.42332139e-13 9.20855987e-12
 4.77099262e-12]
[4.37255344e-10 3.80065292e-14 8.52546213e-09 2.00000000e+00
 2.00000000e+00]
[5.69647136e-14 3.61290176e-07 2.00000000e+00 2.66479919e-24
 4.30102432e-10]
[3.00000000e+00 2.00000000e+00 1.40093628e-51 3.00000000e+00
 2.00000384e+00]
[2.00000000e+000 3.16873215e-112 2.99999999e+000 4.36651826e-013
 9.99996156e-001]
[3.20694096e-52 2.99999964e+00 2.71114261e-13 1.42580043e-11
 3.67550959e-12]
[4.18932483e-10 5.10728338e-14 8.64444070e-09 2.00000000e+00
 2.00000000e+00]
[5.74378505e-14 3.61061990e-07 2.00000000e+00 6.66304574e-22
 4.29161518e-10]
[3.00000000e+00 2.00000000e+00 3.58924972e-52 3.00000000e+00
 2.00001743e+00]
[2.00000000e+000 9.37459088e-114 2.99999999e+000 3.74235948e-013
 9.99982573e-001]
[6.37100672e-56 2.99999964e+00 9.96937577e-13 1.16204350e-11
 5.81857756e-12]
[4.75908698e-10 4.13989484e-14 9.73214499e-09 2.0

[1.70542856e-10 3.00000000e+00 3.96386259e-09 2.00000000e+00
 2.00000000e+00]
[2.28235425e-15 5.03265385e-23 2.00000000e+00 0.00000000e+00
 2.87907173e-20]
[3.00000000e+00 4.26972155e-16 2.41279136e-17 3.00000000e+00
 3.00000000e+00]
[2.00000000e+00 2.00000000e+00 3.00000000e+00 1.71671560e-19
 1.33470466e-12]
[1.51349252e-260 6.53441030e-025 5.58336407e-015 5.38716442e-012
 6.68322648e-013]
[1.73254165e-10 3.00000000e+00 3.93692632e-09 2.00000000e+00
 2.00000000e+00]
[2.00157164e-15 5.45325466e-23 2.00000000e+00 0.00000000e+00
 5.37863833e-15]
[3.00000000e+00 4.20213841e-16 5.27222526e-16 3.00000000e+00
 3.00000000e+00]
[2.00000000e+00 2.00000000e+00 3.00000000e+00 1.99218138e-19
 1.66342232e-16]
[2.66567559e-260 9.05393301e-025 7.52877890e-015 6.72360929e-012
 1.80257259e-015]
[1.65434992e-10 3.00000000e+00 3.91817359e-09 2.00000000e+00
 2.00000000e+00]
[1.96767713e-15 3.88003693e-23 2.00000000e+00 0.00000000e+00
 1.44778188e-18]
[3.00000000e+00 3.01162847e-16 8.41175590e-15 3.000000

[3.00000000e+00 2.58463290e-15 2.00000000e+00 2.51083004e-01
 1.56814070e-14]
[2.00000000e+00 2.00000000e+00 4.72627827e-17 2.92138554e+00
 2.99999987e+00]
[4.39448779e-89 5.53709670e-21 3.00000000e+00 1.08307459e-09
 2.00000000e+00]
[2.50855364e-11 3.00000000e+00 1.37187052e-20 1.72759274e-01
 0.00000000e+00]
[1.77386625e-12 4.17947407e-21 3.11502760e-16 1.82724073e+00
 1.28371853e-07]
[3.00000000e+00 1.73526672e-15 2.00000000e+00 7.86144594e-02
 2.99999530e+00]
[2.00000000e+00 2.00000000e+00 4.97679981e-17 2.99408911e+00
 4.69769841e-06]
[2.06812836e-68 3.43969351e-21 3.00000000e+00 2.46349137e-10
 2.33175279e-10]
[2.53567967e-11 3.00000000e+00 1.31361226e-20 8.25847237e-01
 2.00000000e+00]
[1.75399586e-12 3.50510315e-21 2.92414535e-16 1.17415276e+00
 8.05249873e-10]
[3.00000000e+00 1.65867278e-15 2.00000000e+00 5.91088922e-03
 2.99999998e+00]
[2.00000000e+00 2.00000000e+00 4.48708098e-17 2.99980220e+00
 2.38363866e-08]
[1.25174738e-63 1.70526876e-21 3.00000000e+00 1.13718256e-11
 6.

[2.87774370e-60 2.99976029e+00 3.00000000e+00 2.01845494e-11
 4.92303759e-16]
[3.64848147e-11 1.04411048e-14 2.09606404e-19 2.00000000e+00
 2.00000000e+00]
[5.89682989e-018 2.65143502e-004 1.45450308e-016 3.13656655e-288
 4.26909441e-021]
[3. 2. 2. 3. 3.]
[2.00000000e+00 6.68630972e-48 4.74761374e-19 1.15261916e-17
 1.12187207e-17]
[2.21307898e-63 2.99973486e+00 3.00000000e+00 2.63817644e-11
 5.65781202e-16]
[2.27863171e-11 6.67582578e-15 2.33336730e-19 2.00000000e+00
 2.00000000e+00]
[4.62840212e-018 2.92566349e-004 1.48791401e-016 1.00555060e-278
 7.89694555e-021]
[3. 2. 2. 3. 3.]
[2.00000000e+00 1.42929164e-76 4.94603363e-19 1.09895278e-14
 1.32709893e-17]
[3.62038858e-69 2.99970743e+00 3.00000000e+00 4.16931886e-08
 6.66319857e-16]
[2.88241827e-11 6.34387701e-15 2.58897814e-19 2.00000000e+00
 2.00000000e+00]
[4.14744253e-018 3.21238913e-004 1.50335094e-016 8.52446516e-278
 9.40018964e-021]
[3.         2.         2.         2.99999996 3.        ]
[2.00000000e+000 1.76312509e-112 5.3

 1.17694890e-15]
[1.70948550e-12 3.00000000e+00 6.43066306e-15 1.00000004e+00
 2.00000000e+00]
[1.46369053e-14 1.79161267e-19 1.89953559e-16 9.99999957e-01
 9.13728643e-26]
[3.00000000e+00 8.30691841e-16 2.00000000e+00 3.00000000e+00
 3.00000000e+00]
[2.00000000e+00 2.00000000e+00 2.39749509e-14 2.59627345e-13
 8.83145622e-19]
[7.36323972e-78 1.11285159e-17 3.00000000e+00 2.63237802e-10
 1.28951449e-15]
[3.63815091e-12 3.00000000e+00 8.08547678e-15 1.00000001e+00
 2.00000000e+00]
[2.74876089e-14 1.33252778e-19 1.91087661e-16 9.99999988e-01
 1.48565567e-25]
[3.0000000e+00 6.8251565e-16 2.0000000e+00 3.0000000e+00 3.0000000e+00]
[2.00000000e+00 2.00000000e+00 2.72890233e-14 4.32372932e-13
 1.08831493e-15]
[3.22555783e-74 6.94313088e-18 3.00000000e+00 2.75266121e-10
 1.06496548e-12]
[7.63614756e-12 3.00000000e+00 1.01288946e-14 1.00000000e+00
 2.00000000e+00]
[2.85137150e-14 3.20596945e-19 1.86886689e-16 9.99999995e-01
 3.62381678e-23]


In [203]:
# defend_X, defend_lengths = create_hmm_input(events_df7, players=list(range(5)))
# offend_X, offend_lengths = create_hmm_input(events_df7, players=list(range(5, 10)))

In [201]:
# defend_model = hmm.GaussianHMM(n_components=5, covariance_type='diag', n_iter=50, verbose=True)#, random_state=42)
# defend_model.fit(defend_X, defend_lengths)

In [202]:
# offend_model = hmm.GaussianHMM(n_components=5, covariance_type='diag', n_iter=50, verbose=True)#, random_state=42)
# offend_model.fit(offend_X, offend_lengths)

In [138]:
# from hmmlearn import hmm
# model = hmm.GaussianHMM(n_components=5, covariance_type='diag', n_iter=50, verbose=True)
# model.fit(X, lengths)

In [204]:
# cmeans = defend_model.means_
# covars = defend_model.covars_

In [205]:
# state_sequence = defend_model.predict(defend_X, defend_lengths)

In [170]:
d = defend_lengths.reshape(-1, 5)

In [171]:
s = state_sequence.reshape(-1, 5)

In [173]:
s.shape[0]

20571

In [184]:
a = np.arange(10)
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [200]:
a.reshape(5, -1).T

array([[0, 2, 4, 6, 8],
       [1, 3, 5, 7, 9]])

In [186]:
defend_lengths.shape

(820,)

In [196]:
l = defend_lengths.reshape(5, -1)

In [198]:
l[0,:] == l[1,:]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

1) sumsample should only happen all the way until the end, e.g. if velocity is computed after subsample then the direction of the velocity will result in more errors.